# Data Cleaning

In [ ]:
import pandas as pd
import os

## Combining CSVs

In [ ]:
# Directory containing the CSV files
csv_files = [
    "Chevrolet Scrape.csv", "Ford Scrape.csv", "GMC Scrape.csv", 
    "Honda Scrape.csv", "Hyundai Scrape.csv", "Jeep Scrape.csv", 
    "Kia Scrape.csv", "Nissan Scrape.csv", "Subaru Scrape.csv", 
    "Toyota Scrape.csv"]

# Join CSV files
csv_paths = [os.path.join(directory, file) for file in csv_files]
combined_df = pd.concat([pd.read_csv(file) for file in csv_paths], ignore_index=True)

# Save as CSV
combined_df.to_csv("july_21st_raw_data.csv", index=False)
print('Done')

Done


## Delete duplicate rows

In [ ]:
# Drop 'Listing ID' column to identify duplicates, otherwise it wont find duplicates
combined_df_noListing = combined_df.drop(columns=['Listing ID'])

# Identify duplicates
duplicates_mask = combined_df_noListing.duplicated()

# Number of duplicate rows
num_duplicates = duplicates_mask.sum()
print(f"Found {num_duplicates} duplicate rows.")

# Remove duplicate rows from the original DataFrame while maintaining 'Listing ID'
combined_df = combined_df[~duplicates_mask].reset_index(drop=True)

# Print information about the cleaning process
print(f"Rows after cleaning: {combined_df.shape[0]}")
print(f"Rows deleted: {num_duplicates}")

Found 12 duplicate rows.
Rows after cleaning: 83366
Rows deleted: 12


## Delete rows with empty cells

In [ ]:
def delete_rows_with_empty_cells(df):
    """
    Deletes rows in the DataFrame that contain any empty (NaN) cells.
    """
    empty_cells_count = df.isnull().sum().sum()  
    rows_before = df.shape[0]  
    df.dropna(inplace=True)  
    rows_after = df.shape[0]  
    rows_deleted = rows_before - rows_after  
    
    return empty_cells_count, rows_deleted

# Apply the function to the existing DataFrame
empty_cells_count, rows_deleted = delete_rows_with_empty_cells(combined_df)

print(f"Empty cells count: {empty_cells_count}")
print(f"Rows deleted: {rows_deleted}")

Empty cells count: 1374
Rows deleted: 1297


In [ ]:
print('Empty cells left:',combined_df.isnull().sum().sum())

Empty cells left: 0


## Format price (removing decimal)

In [ ]:
combined_df.head(5)

,Listing ID,Trim,Make,Year,Model,Price,Body Style,City,State,Mileage,Stock Type
0,f41e479e-4d07-42ba-850c-9f9dd21a0d2f,lt cloth,chevrolet,2020,traverse,25988.0,suv,Pittsburgh,PA,51375,used
1,04de8d78-9d1a-407d-aa08-4259f598f589,1lt,chevrolet,2016,silverado 1500,23995.0,pickup truck,Tallahassee,FL,98658,used
2,0ceae6e2-43bd-4a6d-9eaf-ac5ae2a7a703,custom,chevrolet,2021,silverado 1500,32980.0,pickup truck,East Hartford,CT,53326,used
3,c4c93e14-6796-41ea-8503-f7bc7c0c319c,1lt,chevrolet,2018,silverado 1500,29250.0,pickup truck,Caledonia,MI,40000,used
4,f6110b86-5210-4e23-9eb8-3c005fba73b4,ls,chevrolet,2022,spark,13000.0,hatchback,St Augustine,FL,47825,used


In [ ]:
# Convert 'price' column to integer, gets rid of float
combined_df['Price'] = combined_df['Price'].astype(int)
combined_df.head(5)

,Listing ID,Trim,Make,Year,Model,Price,Body Style,City,State,Mileage,Stock Type
0,f41e479e-4d07-42ba-850c-9f9dd21a0d2f,lt cloth,chevrolet,2020,traverse,25988,suv,Pittsburgh,PA,51375,used
1,04de8d78-9d1a-407d-aa08-4259f598f589,1lt,chevrolet,2016,silverado 1500,23995,pickup truck,Tallahassee,FL,98658,used
2,0ceae6e2-43bd-4a6d-9eaf-ac5ae2a7a703,custom,chevrolet,2021,silverado 1500,32980,pickup truck,East Hartford,CT,53326,used
3,c4c93e14-6796-41ea-8503-f7bc7c0c319c,1lt,chevrolet,2018,silverado 1500,29250,pickup truck,Caledonia,MI,40000,used
4,f6110b86-5210-4e23-9eb8-3c005fba73b4,ls,chevrolet,2022,spark,13000,hatchback,St Augustine,FL,47825,used


## Model Formating

In [ ]:
# Count all models by letter so i can combine and drop unnecessary models 
count = combined_df[combined_df['Model'].str.startswith('a')]
count['Model'].value_counts()

Model
altima              1422
acadia              1375
accord              1267
ascent               641
armada               293
accord hybrid        164
accent               153
avalon                77
avalon hybrid         31
avalanche             26
ariya                 26
acadia limited        25
azera                 12
accord crosstour       7
aveo                   1
Name: count, dtype: int64

In [ ]:
# Combine models and correct typos for consistency 

# a models
# none

# b models
# none

# c models
combined_df['Model'] = combined_df['Model'].replace('cr_v', 'cr-v') # replaces the model name
combined_df['Model'] = combined_df['Model'].replace('cr_v hybrid', 'cr-v hybrid')
combined_df['Model'] = combined_df['Model'].replace('c_hr', 'c-hr')
combined_df['Trim'] = combined_df.apply(lambda row: f"{row['Trim']} si" if row['Model'] == 'civic si' else row['Trim'], axis=1) # adds value to trim
combined_df['Model'] = combined_df['Model'].replace('civic si', 'civic')
combined_df['Trim'] = combined_df.apply(lambda row: f"{row['Trim']} type r" if row['Model'] == 'civic type r' else row['Trim'], axis=1)
combined_df['Model'] = combined_df['Model'].replace('civic type r', 'civic')
combined_df['Model'] = combined_df['Model'].replace('clarity plug_in hybrid', 'clarity plug-in hybrid')
combined_df['Model'] = combined_df['Model'].replace('corvette stingray', 'corvette') # stingray is base model for c7 and c8
combined_df['Model'] = combined_df['Model'].replace('c_max energi', 'c-max energi')
combined_df['Model'] = combined_df['Model'].replace('c_max hybrid', 'c-max hybrid')
combined_df['Model'] = combined_df['Model'].replace('cr_z', 'cr-z')

# d models
# none

# e models
combined_df['Trim'] = combined_df.apply(lambda row: f"{row['Trim']} n" if row['Model'] == 'elantra n' else row['Trim'], axis=1)
combined_df['Model'] = combined_df['Model'].replace('elantra n', 'elantra')
combined_df['Trim'] = combined_df.apply(lambda row: f"{row['Trim']} max" if row['Model'] == 'expedition max' else row['Trim'], axis=1)
combined_df['Model'] = combined_df['Model'].replace('expedition max', 'expedition')
combined_df['Trim'] = combined_df.apply(lambda row: f"{row['Trim']} super duty" if row['Model'] == 'e350 super duty' else row['Trim'], axis=1)
combined_df['Model'] = combined_df['Model'].replace('e350 super duty', 'e350')

# f models
combined_df['Model'] = combined_df['Model'].replace('f_150', 'f-150')
combined_df['Model'] = combined_df['Model'].replace('f_250', 'f-250')
combined_df['Model'] = combined_df['Model'].replace('f_350', 'f-350')
combined_df['Model'] = combined_df['Model'].replace('f_450', 'f-450')
combined_df['Model'] = combined_df['Model'].replace('f_150 lightning', 'f-150 lightning')
combined_df['Trim'] = combined_df.apply(lambda row: f"{row['Trim']} st" if row['Model'] == 'focus st' else row['Trim'], axis=1)
combined_df['Model'] = combined_df['Model'].replace('focus st', 'focus')
combined_df['Trim'] = combined_df.apply(lambda row: f"{row['Trim']} rs" if row['Model'] == 'focus rs' else row['Trim'], axis=1)
combined_df['Model'] = combined_df['Model'].replace('focus rs', 'focus')

# g models
combined_df['Model'] = combined_df['Model'].replace('gt_r', 'gt-r')

# h models
combined_df['Model'] = combined_df['Model'].replace('hr_v', 'hr-v')

# i models
combined_df['Trim'] = combined_df.apply(lambda row: f"{row['Trim']} sti" if row['Model'] == 'impreza wrx sti' else row['Trim'], axis=1)
combined_df['Model'] = combined_df['Model'].replace('impreza wrx sti', 'impreza wrx')
combined_df['Trim'] = combined_df.apply(lambda row: f"{row['Trim']} outback sport" if row['Model'] == 'impreza outback sport' else row['Trim'], axis=1)
combined_df['Model'] = combined_df['Model'].replace('impreza outback sport', 'impreza')

# j models
# none 

# k models
# none

# l models
# none

# m models
combined_df['Model'] = combined_df['Model'].replace('mustang mach_e', 'mustang mach-e')

# n models
combined_df['Model'] = combined_df['Model'].replace('nv cargo', 'nv cargo nv1500')
combined_df['Model'] = combined_df['Model'].replace('niro plug_in hybrid', 'niro plug-in hybrid')

# o models
combined_df['Model'] = combined_df['Model'].replace('optima plug_in hybrid', 'optima plug-in hybrid')

# p models
combined_df['Model'] = combined_df['Model'].replace('prius plug_in', 'prius plug-in')

# q models
# none

# r models
# none

# s models
combined_df['Model'] = combined_df['Model'].replace('silverado 1500 ld', 'silverado 1500')
combined_df['Model'] = combined_df['Model'].replace('santa fe plug_in hybrid', 'santa fe plug-in hybrid')
combined_df['Model'] = combined_df['Model'].replace('sonata plug_in hybrid', 'sonata plug-in hybrid')
combined_df['Model'] = combined_df['Model'].replace('sorento plug_in hybrid', 'sorento plug-in hybrid')
combined_df['Model'] = combined_df['Model'].replace('sportage plug_in hybrid', 'sportage plug-in hybrid')

# t models
combined_df['Model'] = combined_df['Model'].replace('transit_150', 'transit 150')
combined_df['Model'] = combined_df['Model'].replace('transit_250', 'transit 250')
combined_df['Model'] = combined_df['Model'].replace('transit_350', 'transit 350')
combined_df['Model'] = combined_df['Model'].replace('tucson plug_in hybrid', 'tucson plug-in hybrid')

# u models
# none

# v models
# none

# w models
# none

# x models
# none

# y models
# none

# z models
# none

# 1-9 models
# none

print('Models Formated')

Models Formated


## Trim Formating

In [ ]:
# Group by 'Make', 'Model', and 'Trim' and count occurrences
trim_counts = combined_df.groupby(['Make', 'Model', 'Trim']).size().reset_index(name='Count')

# Sort better readability
trim_counts = trim_counts.sort_values(by=['Make', 'Model', 'Count'], ascending=[True, True, False])

# To print trim counts by make
for make in trim_counts['Make'].unique():
    make_df = trim_counts[trim_counts['Make'] == make]
    print(f"Make: {make}")
    for model in make_df['Model'].unique():
        model_df = make_df[make_df['Model'] == model]
        print(f"  Model: {model}")
        print(model_df[['Trim', 'Count']])
        print('\n')

Make: chevrolet
  Model: avalanche
       Trim  Count
5       ltz      7
1   1500 lt      6
2  1500 ltz      5
4        lt      4
0   1500 ls      3
3        ls      1


  Model: aveo
   Trim  Count
6  5 ls      1


  Model: blazer
       Trim  Count
8       2lt    135
13       rs     68
9       3lt     30
11       lt     16
12  premier     14
7       1lt     12
10     base      1


  Model: blazer ev
   Trim  Count
14  2lt      1
15   rs      1


  Model: bolt euv
               Trim  Count
16           fwd lt     24
17      fwd premier      9
18               lt      7
20          premier      4
19  lt hatchback 4d      1


  Model: bolt ev
               Trim  Count
24           fwd lt     23
25      fwd premier     23
26               lt     18
28          premier      8
21              1lt      6
23          fwd 2lt      5
22          fwd 1lt      4
27  lt hatchback 4d      1


  Model: camaro
                   Trim  Count
34                  2ss    100
30                  1lt   

In [ ]:
# Combine trims for models and correcting typos for consistency and cleaning

# Create a dictionary for trim name replacements
# Use a dictionary with .replace at the end to organize trims easier 

trim_formatting = {
# 1. CHEVEROLET
    # avalanche
    '1500 ltz': 'ltz',
    '1500 lt': 'lt',
    '1500 ls': 'ls',
    # aveo
    '5 ls': 'ls',
    # blazer
    'base': 'lt',
    # bolt euv
    'fwd premier': 'premier',
    'fwd lt': 'lt',
    'lt hatchback 4d': 'lt',
    # bolt ev
    'fwd premier': 'premier',
    'fwd lt': 'lt',
    'fwd 1lt': '1lt',
    'fwd 2lt': '2lt',
    'lt hatchback 4d': 'lt',
    # camaro
    'lt': '1lt',
    'rwd coupe lt1': 'lt1',
    'ss': '1ss',
    'ls': '1ls',
    'rwd convertible lt1': 'lt1',
    'ls coupe 2d': '1ls',
    # colorado
    'lt trail boss': 'trail boss',
    '2wd lt': 'lt',
    'base': 'lt',
    '2wd work truck': 'work truck',
    '4wd z71': 'z71',
    '4wd trail boss': 'trail boss',
    '4wd zr2': 'zr2',
    # corvette
    'stingray 3lt': 'stingray w/3lt',
    '3lt': 'stingray w/3lt',
    'z06 hardtop': 'z06',
    'stingray z51': 'stingray z51',
    'stingray 1lt': 'stingray w/1lt',
    'stingray 2lt': 'stingray w/2lt',
    'zr_1': 'zr1',
    '1lt': 'stingray w/1lt',
    '2lt': 'stingray w/2lt',
    '427': 'base',
    'stingray z51 3lt luxury convertible sport coupe': 'stingray z51',
    'z06 w/3lz': 'z06',
    # cruze
    'ls auto': 'ls',
    'lt auto': 'lt',
    # equinox
    'lt w/1lt': '1lt',
    'awd 4dr rs': 'awd rs',
    # express 3500
    "135''wb low roof minivan/van w/shelving storage system": "lt",
    '3500': 'lt',
    '3500 3dr extended cargo van w/1wt': 'rwd 3500 extended wheelbase wt',
    # impala
    'ls fleet': 'fleet',
    # malibu
    'fwd lt': 'lt',
    'fwd rs': 'rs',
    'fwd 1lt': '1lt',
    'fwd premier': 'premier',
    '4dr sdn lt': 'lt',
    'fwd 2lt': '2lt',
    'l': 'lt',
    # silverado 1500
    'work truck': 'wt',
    'base': 'lt',
    'work truck 1wt': 'wt',
    'work truck 2wt': 'wt',
    # silverado 2500
    'work truck': 'wt',
    # silverado 3500
    'work truck': 'wt',
    # sonic
    'fwd hatchback 1fl 5_door': 'lt',
    'fwd sedan lt': 'sedan lt',
    'fwd sedan premier': 'sedan premier',
    # ss
    '6_speed manual': 'base',
    'na': 'base',
    # suburban
    'z71 sport utility 4d': '4wd z71',
    'z71': '4wd z71',
    # tahoe
    '2wd rst': 'rst',
    '2wd high country': 'high country',
    # traverse
    'lt w/1lt': '1lt',
    # trax
    'fwd activ': 'activ',
    'fwd 1rs': '1rs',
    'fwd 2rs': '2rs',

# 2. FORD
    # c-max hybrid
    'titanium hybrid': 'titanium',
    # edge
    'st line': 'st_line',
    # escape
    'base': 'se',
    # expedition
    'max limited': 'limited max',
    'max platinum': 'platinum max',
    'max xlt': 'xlt max',
    # f-150
    'king_ranch supercrew 5.5_ft. 4wd': 'king ranch',
    'harley_davidson': 'harley davidson',
    '145': 'xlt',
    'roush 5.11 tactical': 'roush',
    'stx supercrew 4wd': 'stx supercrew',
    'supercrew': 'xlt supercrew',
    # f-250
    "4wd supercab 158'' lariat": "lariat",
    'lariat 4wd crew cab 6.75 box': 'lariat',
    'xl 4x2 4dr supercab 8 ft. lb pickup': 'xl',
    # fiesta
    'se 4dr hatchback': 'se',
    'se 4dr sedan': 'se',
    'st 4dr hatchback': 'st',
    # focus
    'st st': 'st',
    'base rs': 'rs',
    'base st': 'st',
    # fusion hybrid
    'hybrid se': 'se',
    'hybrid titanium': 'titanium',
    'hybrid sel': 'sel',
    'se hybrid': 'se',
    # mustang
    'mach 1 fastback': 'mach 1',
    'dark horse fastback': 'dark horse',
    # ranger
    'xlt 2wd supercrew 5 box': 'xlt',
    # transit 250
    '148 wb high roof extended cargo': 'base',
    '148 wb medium roof cargo': 'base',
    'base w/60/40 pass_side cargo_doors': 'base',
    'roof w/rwb': 'base',
    # transit 350
    '350 hd xlt': 'xlt',

# 3. GMC
    # acadia limited
    'base awd 4dr suv': 'limited',
    '4dr limited 3rd row seating': 'limited',
    'base': 'limited',
    'sport utility 4d': 'limited',
    # canyon
    'at4 w/leather': 'at4',
    '4wd at4 w/leather': 'at4',
    'at4 with leather': 'at4',
    '2wd elevation': 'elevation',
    '4wd at4x': 'at4x',
    '2wd elevation standard': 'elevation standard',
    '4wd denali': 'denali',
    'at4 w/cloth': 'at4',
    'at4 with cloth': 'at4',
    # savana 3500
    '3500 3dr cargo van w/ 1wt': 'cargo van',
    'rwd 3500 extended wheelbase ls': 'extended',
    'rwd 3500 regular wheelbase work van': 'work van',
    # sierra 1500
    '4wd crew cab 147': 'slt',
    'preferred group 1': 'slt',
    'slt crewcab 4wd': 'slt',
    # terrain
    'awd at4': 'at4',
    'sl': 'slt',
    # yukon
    'awd at4': 'at4',
    'gmc yukon, at4, loaded, lifted, toyo tires': 'at4',
    # yukon xl
    'awd at4': 'at4',
    '1500 denali': 'denali',
    '1500 slt': 'slt',
    '1500 sle': 'sle',

# 4. HONDA
    # accord
    'lx 4dr sedan cvt': 'lx',
    'touring v6': 'touring',
    'ex_l w/navigation & honda sensing': 'ex_l',
    '4dr i4 cvt sport': 'sport',
    'lx 4dr sedan': 'lx',
    'lx premium': 'lx',
    'ex_l v_6': 'ex_l',
    'sport 4dr sedan (1.5t i4 cvt)': 'sport 1.5t',
    'sport special edition': 'sport',
    # accord hybrid
    'sport_l w/o bsi': 'sport_l',
    # civic
    'ex_l navi': 'ex_l',
    'ex_l w/navi': 'ex_l',
    'base si': 'si',
    'sport touring': 'sport',
    'touring type r': 'type r',
    'base (m6) type r': 'type r',
    'limited edition type r': 'type r',
    # cr-v
    '2wd ex': 'ex',
    '2wd ex_l': 'ex_l',
    'ex_l awd': 'awd ex_l',
    'ex_l 2wd': 'ex_l',
    'lx awd': 'awd lx',
    'ex awd': 'awd ex',
    '2wd special edition': 'special edition',
    'ex 2wd': 'ex',
    '2wd touring': 'touring',
    'lx 2wd': 'lx',
    '2wd lx': 'lx',
    'ex_l w/navigation': 'ex_l',
    'ex awd 4dr suv': 'awd ex',
    'ex_l sport utility 4d': 'ex_l',
    'sport_l': 'sport',
    'touring awd 4dr suv': 'awd touring',
    # cr-v hybrid
    'hybrid sport touring': 'sport touring',
    'hybrid sport': 'sport',
    'hybrid touring': 'touring',
    'hybrid ex': 'ex',
    'hybrid ex_l': 'ex_l',
    'hybrid sport_l': 'sport_l',
    'hybrid lx': 'ex_l',
    'sport touring awd': 'sport touring',
    # cr-z
    'ex w/navi 2dr hatchback cvt': 'ex',
    # hr-v
    '2wd sport': 'sport',
    'sport awd cvt': 'awd sport',
    'ex_l w/navigation': 'ex_l',
    # insight
    'ex 4dr hatchback': 'ex',
    # odyssey
    'ex_l w/navigation/res': 'ex_l',
    'ex_l w/navi/res': 'ex_l',
    # passport
    '2wd ex_l': 'ex_l',
    '2wd sport': 'sport',
    '2wd touring': 'touring',
    'awd black': 'black edition',
    'ex l awd 4dr suv': 'awd ex_l',
    'trailsport awd': 'awd trailsport',
    # pilot
    '2wd special edition': 'special edition',
    '2wd ex_l': 'ex_l',
    '2wd sport': 'sport',
    'touring 8_passenger': 'touring 8 passenger',
    'touring 7_passenger': 'touring',
    'awd ex_l 7 passenger': 'awd ex_l',
    '2wd ex_l 7 passenger': 'ex_l',
    '2wd ex': 'ex',
    '2wd ex_l 8 passenger': 'ex_l 8 passenger',
    '2wd lx': 'lx',
    # ridgeline
    'black': 'black edition',

# 5. HYUNDAI
    # accent
    'se sedan 4d': 'se',
    # elantra
    'n_line': 'n line',
    'se value edition': 'se',
    'automatic n': 'n',
    'base n': 'n',
    'manual n': 'n',
    # equus
    'signature 4dr sedan': 'signature',
    # genesis
    '3.8l': '3.8',
    # kona n
    'fwd': 'base',
    # palisade
    'essential fwd': 'essential',
    'limited sport utility 4d': 'limited',
    # santa cruz
    '2.5l sel': 'sel',
    '2.5t limited': 'limited',
    'preferred awd with trend package': 'xrt awd',
    # santa fe
    'calligraphy awd': 'calligraphy',
    'sel 2.0': 'sel',
    'sel awd': 'sel',
    # santa fe sport
    '2.0t': '2.0l turbo',
    'ultimate': '2.0l turbo ultimate',
    # sonata
    'sel+': 'sel plus',
    'sport+': 'sport plus',
    # tucson
    'limited awd': 'limited',
    'sel awd 4dr suv': 'sel',
    'value awd 4dr suv': 'value',
    # tucson hybrid
    'sel convenience awd': 'sel convenience',
    # veloster
    're:flex w/black': 're:flex',
    # veloster n
    'n': 'base',
    # venue
    'sel ivt': 'sel',

# 6. JEEP
    # cherokee
    '80th anniversary fwd': '80th anniversary',
    'latitude fwd': 'latitude',
    'limited fwd': 'limited',
    '75th anniversary edition 4x4': '75th anniversary 4x4',
    '75th anniversary edition': '75th anniversary',
    '75th anniversary edition fwd': '75th anniversary',
    'altitude fwd': 'altitude',
    # compass
    '80th special edition': '80th anniversary',
    'limited fwd': 'limited',
    'all_new latitude': 'latitude',
    'all_new sport': 'sport',
    '75th anniversary edition 4x4': '75th anniversary 4x4',
    'latitude fwd': 'latitude',
    'altitude fwd': 'altitude',
    'latitude lux fwd': 'latitude lux',
    'sun and wheel fwd': 'sun and wheel',
    # gladiator
    "willy's": 'willys',
    # grand cherokee
    'limited 4x2': 'limited',
    '75th anniversary edition': '75th anniversary',
    '80th anniversary 4x2': '80th anniversary',
    'freedom 4x2': 'freedom',
    'laredo e 4x2': 'laredo e',
    'altitude x 4x2': 'altitude x',
    'laredo 4x2': 'laredo',
    'laredo x 4x2': 'laredo x',
    # grand cherokee 4xe
    'anniversary edition': '30th anniversary',
    # grand cherokee l
    'altitude 4x2': 'altitude',
    # grand cherokee wk
    'limited 4x2': 'limited',
    'laredo x 4x2': 'laredo x',
    # grand wagoneer
    'series ii 4wd': 'series ii 4x4',
    'series ii 4wd grand wagoneer twin turbo': 'series ii 4x4',
    # patriot
    'high altitude edition': 'high altitude',
    'altitude edition': 'altitude',
    '75th anniversary edition': '75th anniversary',
    '75th anniversary edition 4x4': '75th anniversary 4x4',
    # renegade
    'jeepster fwd': 'jeepster',
    'latitude fwd': 'latitude',
    # wagoneer
    'series ii 4x2': 'series ii',
    'series iii 4x2': 'series iii',
    'series ii carbide 4x2': 'series ii carbide',
    'series iii 2wd': 'series iii',
    # wagoneer l
    'series iii 4x2': 'series iii',
    # wrangler (4x4 or 4wd is standard, only 07-10 was available in 2wd) (the 4 vs 2 door were all one and two entries)
    '4_door sahara 4x4': 'sahara',
    'willy wheeler 4x4': 'willys wheeler',
    '4_door sahara': 'sahara',
    'all_new unlimited sport s': 'unlimited sport s',
    '4_door high altitude 4x4': 'high altitude',
    '4_door rubicon 4x4': 'rubicon',
    '2_door freedom 4x4': 'freedom',
    '2_door rubicon 4x4': 'rubicon',
    '2_door sport 4x4': 'sport',
    '2_door sport s 4x4': 'sport s',
    '4_door high tide 4x4': 'high tide',
    '4_door rubicon 392 4x4': 'rubicon 392',
    '4_door sahara altitude 4x4': 'sahara altitude',
    '4_door sport altitude 4x4': 'sport altitude',
    '4_door sport s 4x4': 'sport s',
    '4_door willys 4x4': 'willys',
    'all_new sport s': 'sport s',
    'all_new unlimited rubicon': 'unlimited rubicon',
    'all_new unlimited sahara': 'unlimited sahara',
    'sahara 4wd': 'sahara',
    "willy's": "willys",
    'sport 4x4': 'sport',
    'islander 4x4': 'islander',
    'sport s 4x4': 'sport s',
    'willys 4x4': 'willys',
    'rubicon 4x4': 'rubicon',
    'smoky mountain 4x4': 'smoky mountain',
    'willys sport 4x4': 'willys sport',
    'winter 4x4': 'winter',
    # wrangler 4xe
    '4x4': 'rubicon',
    'rubicon 4x4': 'rubicon',
    # wrangler jk
    'sahara 4x4': 'sahara',
    'sport 4x4': 'sport',
    # wrangler unlimited
    'willys 4x4': 'willys',
    'sport s 4x4': 'sport s',
    'rubicon 4x4': 'rubicon',
    'sahara 4x4': 'sahara',
    'freedom 4x4': 'freedom',
    'islander 4x4': 'islander',
    '80th anniversary 4x4': '80th anniversary',
    'sport altitude 4x4': 'sport altitude',
    'moab 4x4': 'moab',
    'high altitude 4x4': 'high altitude',
    'rubicon 4x4': 'rubicon',
    'sport 4x4': 'sport',
    'willy wheeler 4x4': 'willys wheeler',
    'black and tan 4x4': 'black and tan',
    'high tide 4x4': 'high tide',
    'sahara altitude 4x4': 'sahara altitude',
    'willys sport 4x4': 'willys sport',
    '75th anniversary edition 4x4': '75th anniversary',
    'north edition 4x4': 'north edition',
    'rubicon recon 4x4': 'rubicon recon',
    'smoky mountain 4x4': 'smoky mountain',
    'unlimited rubicon 4x4': 'unlimited rubicon',
    'unlimited sahara 4x4': 'unlimited sahara',
    # wrangler unlimited 4xe
    'unlimited sahara': 'sahara',
    'unlimited sahara 4x4': 'sahara',
    'unlimited rubicon': 'rubicon',
    'unlimited sahara high altitude': 'high altitude',
    'unlimited high altitude 4x4': 'high altitude',
    'unlimited rubicon 4x4': 'rubicon',

# 7. KIA
    # cadenza
    '4dr sdn premium': 'premium',
    # ev6
    'light long range': 'light',
    # forte
    'ex w/ gray interior pkg, leatherette, heated seats': 'ex',
    'fe 6m': 'fe',
    'gt w/  smart trunk, cloth/leatherette,': 'gt_line',
    'gt_line w/ cloth/leatherette seats, bluetooth,': 'gt_line',
    # k5
    'gt 4dr sedan': 'gt_line',
    'lxs certified pre_owned': 'lxs',
    'gt_line**still looks new prem pk': 'gt_line',
    'lxs w/ remote start, heated side mirrors,': 'lxs',
    # k900
    'luxury 3.8l': 'luxury',
    # niro
    'lxs special edition': 'lxs',
    # niro ev
    'ex premium 4dr crossover': 'ex premium',
    # niro plug-in hybrid
    'ex fwd': 'ex',
    'sx touring': 'sx',
    # seltos
    'nightfall': 'nightfall edition',
    'certified one owner s': 's',
    'lx ivt awd': 'lx',
    # sorento
    'ex v6': 'ex',
    'sx prestige x_line': 'x_line sx prestige',
    'ex sport': 'ex',
    '2.4l lx': 'lx',
    # sorento hybrid
    'ex awd': 'ex',
    'hybrid ex awd': 'ex',
    # sorento plug-in hybrid
    'gt line 4dr crossover (2.0l i4)': 'gt_line',
    'plus': '+',
    # sportage
    'base': 'lx',
    'ex ** like new**': 'ex',
    'nightfall awd': 'nightfall',
    'sx turbo**cf cert 1 owner': 'sx',
    'x_line w/ premium pkg, sunroof, power': 'x_line',
    'x_pro prestigeawd': 'x_pro prestige',
    # sportage hyrbid
    'sx_prestige': 'sx prestige',
    # stinger
    'base awd 18 wheels': 'base',
    # telluride
    'sx_prestige x_pro': 'sx prestige x_pro',
    'sx_prestige x_line': 'sx prestige x_line',
    'sx_prestige': 'sx prestige',
    'ex awd': 'ex',
    'ex premium pkg': 'ex',
    'ex********one owner******': 'ex',
    'ex********super clean 1 owner******': 'ex',
    'lx awd': 'lx',
    's awd': 's',
    'sx 4dr suv': 'sx',
    'sx w/ leather, 3rd row, 2nd row captains': 'sx',
    'x_line ex': 'ex x_line',

# 8. NISSAN
    # 370z
    'sport tech': 'sport',
    'nismo tech': 'nismo',
    # altima
    'sr fwd': 'sr',
    's fwd': 's',
    'sr vc_turbo fwd': 'sr vc-turbo',
    'platinum fwd': 'platinum',
    '2.0 platinum': 'platinum',
    '2.5 s 4dr sedan': '2.5 s',
    '2.5 sr sedan 4d': '2.5 sr',
    'platinum vc_turbo fwd': 'platinum vc-turbo',
    # armada
    'sl 2wd': 'sl',
    'platinum 2wd': 'platinum',
    'sv 2wd': 'sv',
    # cube
    '5dr wgn i4 cvt 1.8 s': '1.8 s',
    '1.8s': '1.8 s',
    # frontier
    'sv_i4': 'sv',
    # gt-r
    'premium dual_clutch 6_speed transmission': 'premium',
    'premium awd 2dr coupe': 'premium',
    # juke
    's 4dr crossover': 's',
    'sl sport utility 4d': 'sl',
    'sv awd 4dr crossover': 'sv',
    'nismo': 'nismo rs',
    # leaf
    'sv plus 60 kwh': 'sv plus 62 kwh',
    'sv 4dr hatchback': 'sv',
    # murano
    'sv fwd': 'sv',
    'sl fwd': 'sl',
    's fwd': 's',
    'platinum fwd': 'platinum',
    # nv cargo nv1500
    '2500 s': 's v6',
    # nv cargo nv2500 hd
    'sv': 'sv v6',
    'sv v6/sv v8': 'sv v8',
    # nv passenger nv3500 hd
    'sl': 'sl v8',
    # pathfinder
    'sv 2wd': 'sv',
    'sl 2wd': 'sl',
    'sl fwd': 'sl',
    'platinum fwd': 'platinum',
    'platinum 2wd': 'platinum',
    'sv fwd': 'sv',
    's 2wd': 's',
    's fwd': 's',
    'sl awd 4dr suv': 'sl 4wd',
    # quest
    '3.5 sl': 'sl',
    # sentra
    '4dr sdn i4 cvt s': 's',
    # xterra
    'off_road': 'off road',
    's 4x2 4dr suv': 's',

# 9. SUBARU
    # brz
    'limited 2dr coupe 6a': 'limited',
    'limited 2dr coupe 6m': 'limited',
    # crosstrek
    'base trim level': 'base',
    '2.0i premium heated seats, one owner': '2.0i premium',
    'limited cvt': 'limited',
    'limited, heated leather seats, eyesight': 'limited',
    'premium manual': 'premium',
    # crosstrek hybrid
    'hybrid': 'base',
    # forester
    '2.5 x premium': '2.5x premium',
    '2.5 x limited': '2.5x limited',
    '2.5 x': '2.5x',
    '2.5 xt limited': '2.5xt limited',
    'base trim level': 'base',
    'premium cvt': 'premium',
    'limited powered tail gate, navigation': 'limited',
    # impreza
    '2.5 i premium': '2.5i premium',
    '2.5 i': '2.5i',
    '5_door cvt': '5_door',
    'base outback sport': 'outback sport',
    'base trim level': 'base',
    'premium cvt': 'premium',
    'limited powered tail gate, navigation': 'limited',
    # impreza wrx
    'base sti': 'sti',
    'sti limited sti': 'sti limited',
    # legacy
    '2.5 i premium': '2.5i premium',
    '2.5 i': '2.5i',
    '2.5i prem awp/pwr moon': '2.5i premium',
    # outback
    '2.5 i premium': '2.5i premium',
    '2.5 i limited': '2.5i limited',
    '2.5 i': '2.5i',
    '3.6 r limited': '3.6r limited',
    '2.5i prem': '2.5i premium',
    'limited awd wagon': 'limited',
    'limited, heated steering, navigation': 'limited',
    'premium w/ #13 op pkg': 'premium',
    'limited, new tires, navigation': 'limited',
    'touring xt cvt': 'touring xt',
    # wrx
    'base trim level': 'base',
    # wrx sti
    'sti': 'base',
    'sti limited': 'limited',

# 10. TOYOTA
    # 4runner
    'trail': 'trail special edition',
    '40th anniversary': '40th anniversary special edition',
    'nightshade': 'nightshade special edition',
    # c-hr
    'nightshade': 'nightshade edition',
    'xle 4dr crossover': 'xle',
    # camry
    'se w/special edition pkg': 'se night shade',
    'trd v6 w/two_tone midnight black roof': 'trd v6',
    'xle v6': 'xle',
    # camry hybrid
    'hybrid xse': 'xse',
    # camry hybrid
    's w/special edition pkg': 's special edition',
    # corolla cross hybrid
    'hybrid xse': 'xse',
    'hybrid se': 'se',
    # corolla hybrid
    'hybrid le': 'le',
    # highlander
    'le i4': 'le',
    'base v6 (a5)': 'base v6',
    # prius
    'ii': 'two',
    'iii': 'three',
    'iv': 'four',
    'v': 'five',
    'i': 'one',
    'persona series special edition': 'persona series',
    '5dr hb persona (natl)': 'persona series',
    # rav4
    'trd off_road': 'trd off road',
    # rav4 hybrid
    'hybrid xse': 'xse',
    'hybrid limited': 'limited',
    # sienna
    'xle 7 passenger': 'xle',
    'xse 7 passenger': 'xse',
    'platinum 7 passenger': 'platinum',
    'limited 7_passenger': 'limited',
    'ce': 'se',
    'ltd': 'limited',
    # supra
    '2': '2.0',
    '3': '3.0',
    'a91_mt edition': 'a91 edition',
    # tacoma
    'trd off_road': 'trd off road',
    '22 toyo tacoma': 'base',
    'v6': 'base',
    # tundra
    '1794': '1794 edition',
    'trd pro 5.7l v8': 'trd pro',
    'base': 'sr',
    'trd off_road _ lift + topper + bumper + whls/tires': 'trd pro',
    # tundra hybrid
    '1794': '1794 edition',
}


# Apply the replacements
combined_df['Trim'] = combined_df['Trim'].replace(trim_formatting)
print('Trim is formatted')

Trim is formatted


In [ ]:
combined_df.to_csv('car_data_cleaned.csv', index=False)
print('saved', datetime.fromtimestamp(time.time()))

saved 2024-08-06 14:41:23.344602
